In [8]:
!pip install transformers --upgrade
!pip install datasets
!pip install trl[peft] --upgrade
!pip install -U git+https://github.com/huggingface/trl
!pip install bitsandbytes loralib
!pip install wandb -U
!pip install hf_transfer
!pip install sentencepiece



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
  Using cached trl-0.9.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.9.6.dev0
    Uninstalling trl-0.9.6.dev0:
      Successfully uninstalled trl-0.9.6.dev0

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/huggingface/trl to /tmp/pip-req-build-jz5zqd8p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl /tmp/pip-req-build-jz5zqd8p
  Resolved https://github.com/huggingface/trl to commit 82b07d6b0169bb8150f2fa4ee0a58b678d597163
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

In [9]:
%env HF_HUB_ENABLE_HF_TRANSFER=True
%env WANDB_PROJECT=LLM-Training-Course
%env WANDB_RUN_ID=KTO
%env WANDB_NOTEBOOK_NAME={__vsc_ipynb_file__}

env: HF_HUB_ENABLE_HF_TRANSFER=True
env: WANDB_PROJECT=LLM-Training-Course
env: WANDB_RUN_ID=KTO
env: WANDB_NOTEBOOK_NAME=/root/llm-training-course/04-dpo-orpo/02-easier-data-curation-with-kto.ipynb


In [10]:
import wandb
wandb.login()

wandb: Currently logged in as: gal-apriori. Use `wandb login --relogin` to force relogin


True

In [11]:
import sys
sys.path.append('/root/llm-training-course/')

In [12]:
from datasets import load_dataset

train_ds, eval_ds = load_dataset("mlabonne/orpo-dpo-mix-40k", split=["train[:20%]","train[20%:25%]"])

In [13]:
train_ds

Dataset({
    features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
    num_rows: 8849
})

In [14]:
train_ds = train_ds.map(lambda x: { "messages": [{"role":"system", "content": x["prompt"] }] + x["chosen"] })
eval_ds = eval_ds.map(lambda x: { "messages": [{"role":"system", "content": x["prompt"] }] + x["chosen"] })

In [15]:
from transformers import AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [16]:
from datasets import Dataset

kto_train_ds = {"prompt":[], "label":[], "completion": []}
for item in train_ds:
    kto_train_ds["prompt"].append(item["prompt"])
    kto_train_ds["completion"].append(tokenizer.apply_chat_template(item["chosen"], tokenize=False))
    kto_train_ds["label"].append(True)
    kto_train_ds["prompt"].append(item["prompt"])
    kto_train_ds["completion"].append(tokenizer.apply_chat_template(item["rejected"], tokenize=False))
    kto_train_ds["label"].append(False)
kto_train_ds =  Dataset.from_dict(kto_train_ds)

In [18]:
from datasets import Dataset

kto_eval_ds = {"prompt":[], "label":[], "completion": []}
for item in eval_ds:
    kto_eval_ds["prompt"].append(item["prompt"])
    kto_eval_ds["completion"].append(tokenizer.apply_chat_template(item["chosen"], tokenize=False))
    kto_eval_ds["label"].append(True)
    kto_eval_ds["prompt"].append(item["prompt"])
    kto_eval_ds["completion"].append(tokenizer.apply_chat_template(item["rejected"], tokenize=False))
    kto_eval_ds["label"].append(False)
kto_eval_ds =  Dataset.from_dict(kto_eval_ds)

In [19]:
import torch
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


In [20]:
from helpers import stream_responses_for_sample
from transformers import GenerationConfig

generation_config =  GenerationConfig(max_new_tokens=75)
sample_conversations = [
    [{"role": "user", "content": "What is the capital of France?"}],
    [{"role": "user", "content": "Write me a javascript function that check if string is palindrome."}],
    [{"role": "user", "content": "Given x^2=36-4 what is x?"}]
]
stream_responses_for_sample(model, tokenizer, sample_conversations,generation_config)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] What is the capital of France?[/INST] The capital of France is Paris. It is one of the most famous cities in the world, known for its rich history, art, culture, and landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also the political, economic, and cultural center of France.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Write me a javascript function that check if string is palindrome.[/INST] Here is a simple JavaScript function that checks if a string is a palindrome. This function will ignore spaces, punctuation, and convert the string to lowercase for comparison.

```javascript
function isPalindrome(str) {
   const re = /[\W_]/g;
   let cleanStr = str.toLowerCase().replace(


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Given x^2=36-4 what is x?[/INST] To find the value of x, you first need to rearrange the equation to isolate x. Here's how:

1. Add 4 to both sides of the equation:

x^2 + 4 = 36

2. Then, subtract 4 from both sides:

x^2 + 4 -


In [21]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj","down_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm"],
    bias="none",
    task_type="CAUSAL_LM"
)

In [22]:
from trl import KTOTrainer, KTOConfig
import os
training_args = KTOConfig(
    output_dir=os.getenv("WANDB_RUN_ID") + "_KTO" ,
    report_to="wandb",
    num_train_epochs=1.0,
    do_train=True,
    do_eval=True,
    log_level="debug",
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=4,
    lr_scheduler_type="constant",
    bf16=True,
    warmup_steps=0,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=0.2,
    max_grad_norm=.3,
    learning_rate=1e-6,
    beta=0.1,
    desirable_weight=1.0,
    undesirable_weight=1.0,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
kto_trainer = KTOTrainer(
    model,
    peft_config=peft_config,
    args=training_args,
    train_dataset=kto_train_ds,
    eval_dataset=kto_eval_ds,
    tokenizer=tokenizer,
)
kto_trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:464: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the KTOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:474: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the KTOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:504: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your KTOConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Processing tokenized eval KL dataset: 100%|██████████| 4424/4424 [00:04<00:00, 1054.25 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum o

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx

Step,Training Loss,Validation Loss,Rewards/chosen,Logps/chosen,Rewards/rejected,Logps/rejected,Rewards/margins,Logits/chosen,Logits/rejected,Kl
111,0.499200,0.493075,-0.230448,-428.068801,-0.287806,-443.023480,0.057358,-109491096.000000,-110238656.000000,0.016513
222,0.498900,0.486069,-0.441539,-430.179730,-0.565697,-445.802385,0.124159,-109346280.000000,-110095224.000000,0.001581
333,0.498800,0.478953,-0.602162,-431.785884,-0.803372,-448.179137,0.201211,-109120064.000000,-109864672.000000,0.000000
444,0.493500,0.477345,-0.633423,-432.098525,-0.859937,-448.744773,0.226514,-109117304.000000,-109778808.000000,0.005444



***** Running Evaluation *****
  Num examples = 4424
  Batch size = 4
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)

***** Running Evaluation *****
  Num examples = 4424
  Batch size = 4

***** Running Evaluation *****
  Num examples = 4424
  Batch size = 4

***** Running Evaluation *****
  Num examples = 4424
  Batch size = 4
Saving model checkpoint to KTO_KTO/checkpoint-500
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/d79d1742f78eb0cc788c11e5b41a7539d7cb56ef/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "init

TrainOutput(global_step=553, training_loss=0.49476417969191483, metrics={'train_runtime': 20218.9051, 'train_samples_per_second': 0.875, 'train_steps_per_second': 0.027, 'total_flos': 0.0, 'train_loss': 0.49476417969191483, 'epoch': 0.999774011299435})